In [1]:
# Import the research library 
import logging
import benchmarklib
from benchmarklib import BenchmarkDatabase, CompileType, GroverRunner, GroverConfig, calculate_grover_iterations

benchmarklib.setup_logging(logging.INFO)

from sat import ThreeSat, ThreeSatTrial

db = BenchmarkDatabase("3SAT.db", ThreeSat, ThreeSatTrial)

benchmarklib.quantum_trials - INFO - Database initialized: 3SAT.db (3SAT)


In [2]:
# Load Qiskit
from qiskit_ibm_runtime import QiskitRuntimeService, Batch
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
API_INSTANCE = os.getenv("API_INSTANCE", None)
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN, instance=API_INSTANCE)
backend = service.backend(name="ibm_rensselaer")

/tmp/ipykernel_11564/2424275111.py:9: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN, instance=API_INSTANCE)


In [3]:
# Configure Grover Benchmark
config = GroverConfig(shots=10**4, optimization_level=3)
# error correction
config.sampler_options.dynamical_decoupling.enable = True
runner = GroverRunner(db_manager=db, service=service, backend=backend, config=config)

benchmarklib.grover - INFO - GroverRunner initialized for 3SAT problems


In [4]:
# problem parameters
num_vars_range = range(3, 11)

In [ ]:
for compile_type in [CompileType.CLASSICAL_FUNCTION, CompileType.XAG]:
    runner.start_batch(compile_type)
    
    for num_vars in num_vars_range:
        # Accumulate circuits for this num_vars group
        for problem in db.find_problem_instances(size_filters={'num_vars': num_vars}, ...):
            optimal_grover_iters = calculate_grover_iterations(len(problem.solutions), 2**num_vars)
            for grover_iter in range(1, optimal_grover_iters + 1):
                runner.run_grover_benchmark(
                    problem_instance=problem,
                    compile_type=compile_type, 
                    grover_iterations=grover_iter,
                )
        
        # Submit all circuits for this num_vars as one job
        job_id = runner.submit_job()
        print(f"Submitted job {job_id} for {compile_type.value}, {num_vars} vars")
    
    runner.finish_batch()

benchmarklib.grover - INFO - Problem Instance ID: 1778
base_primitive._get_mode_service_backend:WARNING:2025-06-15 09:56:18,373: A backend was passed in as the mode but a session context manager is open so this job will run inside this session/batch instead of in job mode.
/home/nels/.miniconda3/envs/tweedledum/lib/python3.10/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:875: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)
benchmarklib.grover - INFO - Submitted job d17d20r3grvg008jmv7g to ibm_rensselaer
benchmarklib.grover - INFO - Saved trial 4130
benchmarklib.grover - INFO - Problem Instance ID: 1778
base_primitive._get_mode_service_backend:WARNING:2025-06-15 09:56:20,370: A backend was passed in as the mode but a session context manager is open so this job will run 